In [1]:
from ovcg import *
from fesol import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor

In [2]:
k=20
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

use_relax = True
use_solid_steady = True

# tolerance setting
tol = 1e-6
if tol > 1e-1:
    print('The tolerance is too large, this may lead to instability or low quality results')

In [3]:
# since this is a matching interface, we can create
# exact interface nodes mappings.
# this step is done offline
inodes = 100
s2f = np.zeros(inodes, dtype=int)
f2s = s2f.copy()
s2f[1] = inodes-1
start = inodes-2
for i in range(inodes-2):
    s2f[i+2] = start
    start = start - 1
count = 0
for i in range(inodes):
    f2s[s2f[i]] = i

In [4]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i.cmd' % k)
solverF.init_grid()
solverF.init_temperature_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFDSNst%i.pvd' % k)

In [5]:
# solid solver
fixed_temp_tag = 2
adiabatic_tag = 3
interface_tag = 4

kappa = 100.0
rho = 1.0
Cp = 100.0
T_bot = 310.0

solverS = HeatSolver(kappa=kappa, rho=rho, Cp=Cp, steady=use_solid_steady, time_order=2)
solverS.output_file = 'solid_outputs/resFDSNst%i.pvd' % k
solverS.load_mesh('solid_inputs/solid_plate.msh')
solverS.set_initial_condition(T_bot)
solverS.define_adiabatic_bd(tag=adiabatic_tag)
solverS.define_const_temperature_bd(tag=fixed_temp_tag, value=T_bot)
solverS.define_flux_interface(tag=interface_tag)
solverS.init_solver()

In [6]:
# coupling data and setups
fluxF = InterfaceData(size=inodes, value=0.0)
fluxS = InterfaceData(size=inodes, value=0.0)
tempF = InterfaceData(size=inodes, value=300.0)
tempS = InterfaceData(size=inodes, value=T_bot)
thetaT = InterfaceData(size=inodes, value=0.0)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=tol)
conv_solution = RelativeCovergenceMonitor(tol=1e-5)

# maximum pc steps allowed
max_pc_steps = 200

In [7]:
# coupling timing information, use fixed time step for now
# Tfinal = 1.0
# N = 100
# dt = Tfinal/N
dt = 0.05
Nmax = 8000
t = 0.0
avg_pc_iterations = 0.0
flog = open('FDSNst%i.log' % k, mode='w')
flog.write('Fluid Dirichlet with solid Neumann setting, study case: %i\n' % k)
flog.close()
flog = open('FDSNst%i.log' % k, mode='a')

tf = 0.5*dt

In [8]:
# back up solutions
solverF.backup_current_solutions()
solverS.backup()
first_pc_counts = 0
while True:
    first_pc_counts = first_pc_counts + 1
    tempF.backup()
    # first advance the solid by half time
    solverF.advance(t=tf, dt=0.5*dt)
    # get the half time step flux
    fluxF.curr[:] = solverF.get_heat_flux_interface(
        grid=interface_grid,
        side=interface_side,
        axis=interface_axis
    )
    # mapping
    fluxS.curr[:] = fluxF.curr[f2s]
    # update solid flux interface
    solverS.update_flux_interface(interface_tag, fluxS.curr)
    solverS.advance(t=tf, dt=0.5*dt)
    tempS.curr[:] = solverS.get_interface_temperature(tag=interface_tag)
    tempF.curr[:] = tempS.curr[s2f]
    tempF.update_res()
    is_conv = conv_mntr.determine_convergence(tempF)
    if is_conv or first_pc_counts >= max_pc_steps:
        solverF.put_temperature_interface(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis,
            data=tempF.curr
        )
        break
    else:
        if use_relax:
            under_relax.determine_omega(tempF)
            under_relax.update_solution(tempF)
        solverF.put_temperature_interface(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis,
            data=tempF.curr
        )
        solverF.restore_previous_solutions()
        solverS.restore()
    
print(first_pc_counts)
temps_prev = tempS.curr.copy()
fluxf_prev = fluxF.curr.copy()

7


In [9]:
# begin to couple
for step in range(Nmax):
    t += dt
    tf += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup()
    
    # we need to use physically stable previous temperature and flux
    temps_prev[:] = tempS.curr
    fluxf_prev[:] = fluxF.curr
    
    while True:
        pc_counts = pc_counts + 1
        
        # back up previous interface value
        fluxS.backup()
        tempS.backup()
        fluxF.backup()
        
        # advance solid
        solverS.advance(t=t, dt=dt)
        # retrieve solid interface temperature
        tempS.curr[:] = solverS.get_interface_temperature(tag=interface_tag)
        
        # mapping
        tempF.curr[:] = 0.5*(tempS.curr[s2f]+temps_prev[s2f])
        
        solverF.put_temperature_interface(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis,
            data=tempF.curr
        )
        
        # advance fluid
        solverF.advance(t=tf, dt=dt)
        # retrieve fluid interface flux
        fluxF.curr[:] = solverF.get_heat_flux_interface(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # mapping
        fluxS.curr[:] = 0.5*(fluxF.curr[f2s]+fluxf_prev[f2s])
        
        # update residual
        fluxS.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(fluxS)
        if is_conv or pc_counts >= max_pc_steps:
            solverS.update_flux_interface(interface_tag, fluxS.curr)
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            if use_relax:
                under_relax.determine_omega(fluxS)
                under_relax.update_solution(fluxS)
            solverS.update_flux_interface(interface_tag, fluxS.curr)
            solverF.restore_previous_solutions()
            solverS.restore()  

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    avg_pc_iterations += pc_counts
    if step % 50 == 0 or step == Nmax-1:
        solverS.write()
        fluid_ofile.write(solverF, tf)
    thetaT.backup()
    thetaT.curr[:] = (solverF.get_temperature_interface(3,0,1)-300)/10
    thetaT.update_res()
    is_conv = conv_solution.determine_convergence(thetaT)
    if is_conv:
        if step % 50 and step != Nmax-1:
            solverS.write()
            fluid_ofile.write(solverF, t)
        break
    
flog.close()

step=0, pc_iterations=11.
step=1, pc_iterations=6.
step=2, pc_iterations=6.
step=3, pc_iterations=6.
step=4, pc_iterations=6.
step=5, pc_iterations=6.
step=6, pc_iterations=6.
step=7, pc_iterations=6.
step=8, pc_iterations=6.
step=9, pc_iterations=6.
step=10, pc_iterations=6.
step=11, pc_iterations=6.
step=12, pc_iterations=6.
step=13, pc_iterations=6.
step=14, pc_iterations=6.
step=15, pc_iterations=6.
step=16, pc_iterations=6.
step=17, pc_iterations=6.
step=18, pc_iterations=6.
step=19, pc_iterations=6.
step=20, pc_iterations=6.
step=21, pc_iterations=6.
step=22, pc_iterations=6.
step=23, pc_iterations=6.
step=24, pc_iterations=6.
step=25, pc_iterations=6.
step=26, pc_iterations=6.
step=27, pc_iterations=6.
step=28, pc_iterations=6.
step=29, pc_iterations=6.
step=30, pc_iterations=6.
step=31, pc_iterations=6.
step=32, pc_iterations=6.
step=33, pc_iterations=6.
step=34, pc_iterations=6.
step=35, pc_iterations=6.
step=36, pc_iterations=6.
step=37, pc_iterations=6.
step=38, pc_iteration

In [10]:
theta=(solverF.get_temperature_interface(3,0,1)-300)/10
print(theta)
sio.savemat(
    'numeric_res/kFDSNst%i.mat' % k,
    {'theta': theta,
     'avg_pc_its': avg_pc_iterations/(step+1),
     'N': (step+1)
    }
)

[0.91592408 0.92263602 0.92746777 0.93141651 0.93477436 0.93762095
 0.94015781 0.94242828 0.94450365 0.94645661 0.94824614 0.94993191
 0.95150636 0.95300042 0.95440426 0.95574062 0.95700489 0.95821624
 0.95936895 0.96046615 0.96151722 0.96252066 0.96348584 0.96440811
 0.96529032 0.96613641 0.96695238 0.96773506 0.96848943 0.96921418
 0.96991005 0.97058083 0.97122712 0.97184751 0.97244951 0.97302757
 0.97358368 0.97411963 0.97463879 0.97513887 0.97562227 0.97608893
 0.97653855 0.97697265 0.97739178 0.9777967  0.97818726 0.97856527
 0.97892933 0.97928104 0.97961959 0.97994754 0.98026359 0.98056732
 0.98086107 0.9811429  0.98141516 0.98167617 0.98192686 0.98216811
 0.9823987  0.98261955 0.98282997 0.98303162 0.983223   0.9834039
 0.98357622 0.98373741 0.98388937 0.98403104 0.98416247 0.98428377
 0.98439367 0.98449327 0.98458119 0.98465842 0.98472301 0.98477524
 0.98481554 0.98484133 0.98485412 0.98485134 0.98483305 0.98479885
 0.98474672 0.98467537 0.98458325 0.98446983 0.984331   0.98416